In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
from datasets import load_dataset
data = load_dataset("uoft-cs/cifar10")
data

d:\python workspace\Machine_Leaning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [3]:
train_data = data['train']
test_data = data['test']
# Shuffling the training data to make sure the model does not learn the order of the data
train_data = train_data.shuffle(seed=42)
train_data = train_data[:]
test_data = test_data[:]



In [4]:
# Converting the PIL type images to tensors

from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])
# Checking the length of the training data so I can loop through it and transofrm every PIL image to a tensor whic helps me to train the model
print(len(train_data['img']))
for i in range(len(train_data['img'])):
    train_data['img'][i] = transform(train_data['img'][i])
for i in range(len(test_data['img'])):
    test_data['img'][i] = transform(test_data['img'][i])

50000


In [6]:
# Cross checking if the images is converted to tensors or not
train_data['img'][:5]

[tensor([[[0.1255, 0.1255, 0.1294,  ..., 0.8039, 0.6196, 0.6471],
          [0.1333, 0.1451, 0.1451,  ..., 0.7843, 0.6039, 0.6078],
          [0.1529, 0.1608, 0.1608,  ..., 0.6980, 0.5725, 0.5569],
          ...,
          [0.2275, 0.2275, 0.2275,  ..., 0.4863, 0.5059, 0.5333],
          [0.2275, 0.2275, 0.2314,  ..., 0.4863, 0.5020, 0.5216],
          [0.2275, 0.2235, 0.2353,  ..., 0.4863, 0.4941, 0.5098]],
 
         [[0.1098, 0.0980, 0.0941,  ..., 0.6824, 0.5529, 0.5765],
          [0.1255, 0.1137, 0.1098,  ..., 0.6392, 0.5373, 0.5529],
          [0.1412, 0.1255, 0.1216,  ..., 0.5647, 0.4941, 0.5020],
          ...,
          [0.1804, 0.1765, 0.1765,  ..., 0.4039, 0.4235, 0.4471],
          [0.1804, 0.1804, 0.1765,  ..., 0.4039, 0.4196, 0.4392],
          [0.1765, 0.1725, 0.1804,  ..., 0.4000, 0.4157, 0.4275]],
 
         [[0.0980, 0.1059, 0.1137,  ..., 0.4471, 0.2902, 0.3333],
          [0.1098, 0.1216, 0.1294,  ..., 0.3765, 0.2706, 0.2941],
          [0.1294, 0.1373, 0.1412,  ...,

In [10]:
# As now i have all the input data in the form of tensors, I will now convert the labels to tensors as well
train_data['label'] = torch.nn.functional.one_hot(train_data['label'], num_classes=10).float()
test_data['label'] = torch.nn.functional.one_hot(test_data['label'], num_classes=10).float()
print(train_data['label'][:5])

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
print(train_data['img'][0].shape) # Need to check the shape to calculate dimensions for other formulas

torch.Size([3, 32, 32])


In [21]:
# Input data is ready to be fed to the model
# Now I will create a model

class BeautifulModel(torch.nn.Module):
    def __init__(self):
        super(BeautifulModel, self).__init__()
        self.layer1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2) # Covering 2x2 window with a stride of 2
        self.layer2 = torch.nn.Conv2d(6, 16, 5)
        self.connecting_layer = torch.nn.Linear(16 * 5 * 5, 120)
        self.connecting_layer2 = torch.nn.Linear(120, 84)
        self.output_layer = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.layer1(x)))
        x = self.pool(torch.nn.functional.relu(self.layer2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.nn.functional.relu(self.connecting_layer(x))
        x = torch.nn.functional.relu(self.connecting_layer2(x))
        x = self.output_layer(x)
        return x
    

In [22]:
model = BeautifulModel().to(device)
model

BeautifulModel(
  (layer1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (connecting_layer): Linear(in_features=400, out_features=120, bias=True)
  (connecting_layer2): Linear(in_features=120, out_features=84, bias=True)
  (output_layer): Linear(in_features=84, out_features=10, bias=True)
)

In [32]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
batch_size = 32
epochs = 20
for epoch in range(epochs):
    for i in range(len(train_data['img'])):
        inputs = train_data['img'][i:i+batch_size]
        labels = train_data['label'][i:i+batch_size]

        # Insuring that the input and labesl are tensors
        inputs = torch.stack(inputs).to(device)
        labels = torch.tensor(labels).to(device)

        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    print('Training progress')
    print(f'Epoch {epoch}, Loss: {loss.item()}, accuracy: {torch.sum(torch.argmax(output, dim=1) == torch.argmax(labels, dim=1)).item() / batch_size}')
    with torch.no_grad():
        test_inputs = test_data['img']
        test_labels = test_data['label']
        test_inputs = torch.stack(test_inputs).to(device)
        test_labels = torch.tensor(test_labels).to(device)
        test_output = model(test_inputs)
        test_loss = criterion(test_output, test_labels)
        print(f'Epoch {epoch}, Test Loss: {test_loss.item()}, accuracy: {torch.sum(torch.argmax(test_output, dim=1) == torch.argmax(test_labels, dim=1)).item() / len(test_data["img"])}')
print('Model has learned its stuff')

C:\Users\anshp\AppData\Local\Temp\ipykernel_13712\1831480187.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Training progress
Epoch 0, Loss: 0.00020418466010596603, accuracy: 0.03125


C:\Users\anshp\AppData\Local\Temp\ipykernel_13712\1831480187.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels).to(device)


Epoch 0, Test Loss: 3.4087231159210205, accuracy: 0.437
Training progress
Epoch 1, Loss: 0.006442373152822256, accuracy: 0.03125
Epoch 1, Test Loss: 3.1957385540008545, accuracy: 0.4397
Training progress
Epoch 2, Loss: 0.003602564102038741, accuracy: 0.03125
Epoch 2, Test Loss: 2.908565044403076, accuracy: 0.4941


KeyboardInterrupt: 